<a href="https://colab.research.google.com/github/bae0331/DeepLearning/blob/main/VisualChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download visual-chatgpt


In [1]:
!rm -rf visual-chatgpt && git clone https://github.com/microsoft/visual-chatgpt

Cloning into 'visual-chatgpt'...
remote: Enumerating objects: 459, done.
remote: Counting objects: 100% (222/222), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 459 (delta 41), reused 206 (delta 36), pack-reused 237
Receiving objects: 100% (459/459), 33.41 MiB | 28.22 MiB/s, done.
Resolving deltas: 100% (159/159), done.


Go to directory


In [3]:
cd visual-chatgpt/

/content/visual-chatgpt


Install packages


In [4]:
!pip install -r requirements.txt
!pip install git+https://github.com/IDEA-Research/GroundingDINO.git
!pip install git+https://github.com/facebookresearch/segment-anything.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.0/344.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 51.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 24.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 852.0/852.0 kB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 93.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 90.1 MB/s eta 0:00:00
     ━━━━━━━

Set your OPENAI_API_KEY


In [ ]:
%env OPENAI_API_KEY=your_key

Import dependencies

In [ ]:
from visual_chatgpt import *

Run VisualChatGPT

In [ ]:
!mkdir -p checkpoints
# load = "ImageCaptioning_cuda:0,Text2Image_cuda:0"
load = "Text2Box_cuda:0,Segmenting_cuda:0,Inpainting_cuda:0,ImageCaptioning_cuda:0"

load_dict = {e.split('_')[0].strip(): e.split('_')[1].strip() for e in load.split(',')}
bot = ConversationBot(load_dict=load_dict)
with gr.Blocks(css="#chatbot .overflow-y-auto{height:500px}") as demo:
    lang = gr.Radio(choices = ['Chinese','English'], value=None, label='Language')
    chatbot = gr.Chatbot(elem_id="chatbot", label="Visual ChatGPT")
    state = gr.State([])
    with gr.Row(visible=False) as input_raws:
        with gr.Column(scale=0.7):
            txt = gr.Textbox(show_label=False, placeholder="Enter text and press enter, or upload an image").style(
                container=False)
        with gr.Column(scale=0.15, min_width=0):
            clear = gr.Button("Clear️")
        with gr.Column(scale=0.15, min_width=0):
            btn = gr.UploadButton("🖼️", file_types=["image"])
    lang.change(bot.init_agent, [lang], [input_raws, lang, txt, clear])
    txt.submit(bot.run_text, [txt, state], [chatbot, state])
    txt.submit(lambda: "", None, txt)
    btn.upload(bot.run_image, [btn, state, txt, lang], [chatbot, state, txt])
    clear.click(bot.memory.clear)
    clear.click(lambda: [], None, chatbot)
    clear.click(lambda: [], None, state)
    demo.launch(server_name="0.0.0.0", server_port=7860, share=True, debug=True)

In [ ]:
!ps -aux | grep "python"
# !kill -9 7507    